# Modeling the Effect of Weather on Flight Delays at SeaTac
### Authors: Beichen Liang, Max Zhou, Vanely Ruiz, and Will Bowers

## Problem Overview

For our final project, we are trying to answer what effect, if any, the weather has on outgoing flights. More specifically we are interested in the weather in Seattle and its effect on flights leaving SeaTac. We believe this to be an important topic as thousands of people fly through Seattle everyday, and as all of us as residents know Seattle's weather can be both variable and rainy. We hope to inform both the populace of Seattle that travels and the organization of SeaTac with this information, with the idea that travelers can their flights better and SeaTac can analyze and improve their systems to respond better to weather delays.

Flight delays can have costly consequences. It is estimated that in the U.S. alone, flight delays have a 40.7 billion dollar impact. Additionally, all of that time planes spend on the tarmac results in excess fuel being used and more emissions being released (Fleurquin). Flight delays also result in significant disruptions to aviation safety and the decreased traffic results in losses for the airlines (Gao, 68). Flight delays also cause passengers to prefer other airlines if they experience delays with a certain carrier (Tae-Hwee Lee, 277). Causes of flight delays can range from the unpreventable, severe weather (Gao, 68), to the preventable, crew mishaps and flight order rotation (Fleurquin).

The specific question we hope to answer with this data is, __given a particular day at SeaTac International Airport, can we predict the average departure delay, in minutes, for all flights based upon the weather?__ For the purposes of statistical modeling, which we'll cover later, our null hypothesis would be: _there is no relationship between the average daily departure delay at SeaTac and the weather at SeaTac_. Conversely, our alternative hypothesis would be: _there is a relationship between the average daily departure delay at SeaTac and the weather at SeaTac._ Through exploring our data with statistical models, we will be able to either accept or reject our null hypothesis. 

![title](img/plane_departing_seatac.jpg)
<i><center>Plane Departing SeaTac</center></i>
<i><center>Image courtesy of https://news.theregistryps.com/with-growing-region-seatac-prepares-for-expansion/</center></i>

## Data Preparation

For this project we used two data sets: __weather data__ gather by the National Center for Environmental Information (NOAA) and __flight data__ gathered by the Bureau of Transportation Statistics. 


### Weather Data

We used local climatological data gathered by the National Center for Environmental Information (NOAA) as the weather data source for our project. This dataset includes hourly observations made at SeaTac from Dec. 1, 2017 to Nov. 30, 2018 to match the duration of our flight data. [The source for our data can be found here.](https://www.ncdc.noaa.gov/cdo-web/datatools/lcd)

The raw weather data presented several challenges to our analysis. 

Many columns were dominated by null values, though such columns tend to be trivial measurements that are rarely taken by the weather station. We ended up selecting 15 different weather metrics from the dataset which we think would provide a full picture of the weather conditions that might affect flight delays.

Among the observations we selected, some columns are in formats other than floats that our machine learning process can work with. 

Specifically, some columns with numeric values sometimes contain special characters to denote special conditions. For example, the hourly precipitation columns occasionally use the letter "T" to indicate a trace amount of rain that can't be accurately measured. Because the appearance of such values are rare and they can be practically regarded as 0 rainfall, we decided to convert any non-numeric values in those columns to null to be later imputed using forward-filling.

Some columns like "hourly pressure change" use null values to indicate the absence of any changes. This is deduced from the fact that those values are either non-zero numbers or NaN. We filled those NaN values with 0.

There are columns that need to be transformed to new features. The "hourly sky conditions" column contains strings such as "FEW:02 38 BKN:07 190" to describe certain sky conditions. According to the dataset documentation, the two-digit number like "02" and "07" can indicate the thickness of the clouds. We simplified this feature into dummie columns "cloud_0" for no or light cloud, and "cloud_1" for heavily clouded conditions. 

The "hourly wind direction" expresses wind direction in 360 degrees. We think that the wind direction could be an important factor for flights, but we only need a categorical variable that indicates an approximate direction. So we converted the degrees to north, east, south or west, and created dummie column for each.

As mentioned previously, we used forward-filling to handle any null values we had remaining in the dataset. Forward-filling is a process in which the previous entry for that particular feature is used to fill the null value. Forward-filling is useful in cases where the data is on a time scale, as the assumption being made is the weather one hour has a high likelihood as being the same as the previous hour.

Finally, we created a variable _'join_time'_ which was of the format _yyyymmddhh_ such that we could combine the weather for a specific hour at SeaTac with the average hourly flight delay for that hour. 

### Flight data

We aircraft carrier reporting data gathered by the Bureau of Transportation Statistics (BTS) as the flight data source for our project. This dataset included all flights that arrived or departed at a major airport in Washington State from Dec. 1, 2017 to Nov. 30, 2018, with the month of November 2018 being the latest release of this data. [The source for our data can be found here.](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236)

Unlike the weather data, the raw data was relatively easy to work with.

One thing to note was, due to the sheer amount of flight data the BTS had, flight data had to be fetched month by month, with us having a resulting 12 data sets, one for each month. We merged these data sets into one, year long data set. Additionally, to try and limit the amount of unnecessary data being downloaded, we used a state filter on the BTS site specifying only flight arriving or departing in Washington State. 

Once we had the data, we had a few things to do before we could merge it with our weather data. First, we had to filter out all flights that weren't departing from SeaTac. Next, we noticed a column not included in the description of the data set on the BTS site so we dropped that. Once we had solely the flights departing SeaTac, we created a new column, __'join_time'__, which was in the format _yyyymmddhh_, so that we could join with our weather data on the hour. 

With all flights now belonging to a specific hour, we grouped and aggregated the flights so that the _'join__time'_ column was now unique and corresponded to a __'avg_delay'__ column which had the average flight delay for that hour. For the sake of comparison, we also did the hourly average for the BTS reported weather delay, in an attempt to try and gauge how accurate our models would be. We then created a new data set with just the columns __'join_time', 'avg_delay' and 'actual_weather_delay'__. 

At this point we joined both our cleaned weather and flight data to create the data set we used for analysis and modeling. 

## Exploratory Data Analysis



## Statistical Modeling

Here we will utilize statistical approaches to try and model our outcome variable of interest, the average hourly delay of flights departing SeaTac, using our weather data. 

Let us begin with our null hypothesis: _there is no relationship between the average daily departure delay at SeaTac and the weather at SeaTac._ 

Conversly, our alternate hypothesis: _there is a relationship between the average daily departure delay at SeaTac and the weather at SeaTac._

We utilized both a __linear__ and __polynomial__ statistical model to try and assess our null and alternate hypotheses.


### Linear

First, we performed a multivariate linear regression on the dataset. After dropping all irrelevant data fields such as date and hourlyskycondition, we utilized forward selection, which tries to optimize adjusted R-squared value by adding features that helps most one at a time until the score goes down or run out of features, with statsmodel package to select the best model based on the set of possible variables. 
We have the following summary table for the regression.

<img src="img/linear_coeff.JPG" alt="linear regression table" style="width: 500px;"/>

<img src="img/linear_fit.JPG" alt="linear model fit" style="width: 400px;"/>

<img src="img/linear_actual.JPG" alt="linear actual vs predicted" style="width: 800px;"/>

#### Interpretation

Even with the best linear multivariable model, the adjusted R-squared value is 0.065, which means the model only accounts for about 7% of the total variation. The mean absolute error (MAE) is 8.0459 and the mean squared error (MSE) is 181.0914. The low r-squared value is largely because the extreme left skewness of the data, which is not surprising, since most delays won’t last for very long. However, if we consider the general standard for delayed flight, 90.73% of the total flights fall within 15 minutes interval of our model. Given the low R-squared value, we __cannot reject__ the null hypothesis that the delay has nothing to do with the weather. With an interval of 15 minutes, we still cannot reject the null hypothesis since 90.73% is not statistically significant enough to do so.


### Polynomial

To improve our statistical model, we decided to employ a polynomial regression. After dropping all irrelevant data fields such as date and hourlyskycondition, we splitted the dataset into training and testing sets. The training set is used to fit the model while the testing set gives test result on Mean Squared Error (MSE), which indicates if we need further regularization like a Ridge Regression. Moreover, we performed a grid search on the best number of order in the polynomial regression.

<img src="img/poly_fit.JPG" alt="polynomial model fit" style="width: 400px;"/>

<img src="img/polt_actual.JPG" alt="poly actual vs predicted" style="width: 800px;"/>

#### Interpretation

With the use of polynomial regression, the adjusted R-squared value is 0.1304 which means the model accounts for about 13% of the total variation. The mean absolute error (MAE) is 7.9495 and the mean squared error (MSE) is 168.6598. The low r-squared value is again largely because the extreme left skewness of the data, which is not surprising, since most delays won’t last for very long. However, if we consider the general standard for delayed flight, 90.21% of the total flights fall within 15 minutes interval of our model. Given the low R-squared value, again, we __cannot reject__ the null hypothesis that the delay has nothing to do with the weather. With an interval of 15 minutes, we still cannot reject the null hypothesis since 90.21% is not statistically significant enough to do so.


Since neither of our two statistical models can reject the null hypothesis, we switched to machine learning for more accurate models on the unit of analysis. 

## Machine Learning

Another way in which we can model the hourly average flight delays is through a process called machine learning. Machine learning is a process in which you can train an algorithm to make predictions based off of training data, and then validate it with resulting testing data. Machine learning differs from statistical modeling in the sense that you don't necessarily know exactly how the algorithm came to create its model, but you can measure how accurate the resulting model is. There are different types of machine learning algorithms, but since we are trying to predict a continuous variable, the number of minutes the average delay is for flights in that hour, we used algorithms which are known _regressors_. 

For this project, we used two of the machine learning algorithms we learned in class: __K Nearest Neighbors (KNN)__ and __Decision Trees__. A more detailed explanation of KNN can be found [here](https://www.analyticsvidhya.com/blog/2018/03/introduction-k-neighbours-algorithm-clustering/) while a more detailed explanation of decision trees can be found [here](https://medium.com/@chiragsehra42/decision-trees-explained-easily-28f23241248). As a note, all of the tools we used for machine learning come from a wonderful package called __scikit-learn__, whose webpage and subsequent documentation can be found [here](https://scikit-learn.org/stable/index.html).

### Our Machine Learning Process

Machine learning is often described as a process, one that goes above simply just using a different algorithm. Funnily enough, this process for us remained largely the same, with the main difference being which algorithm we've used. In this section, we will detail the process we used to approach our machine learning models.

#### Splitting Data

As mentioned previously, machine learning works by training an algorithm on pre-existing data, of which you know the outcome variable, in our case, the average flight delay per hour. Our data for modeling exists in the form of one large dataset, meaning that we needed to split it into subsequent training and testing subsets. For the best results, it is effective to have the training subset be the majority of the original data, with the testing subset making up a small amount. A typical split would be _80% training data to 20% testing data_. We started off with those values for our split, but also test 85/15, 90/10 and 75/25. It turns out that our initial split, 80/20 resulted in the best predictions so we chose to go with that. 

#### Constructing a Pipeline

As mentioned previously, machine learning is about the process, which means testing and tuning all parts of that process to get the best result. A fantastic tool for doing this is called a __machine learning pipeline__. Essentially what the pipeline does is allows the data scientist to input the different parameters and hyperparameters they think will have an effect on the outcome, and the pipeline runs through all possible combinations of those settings, finding the most optimized algorithm. For our specific pipeline we included three things: a __data scaler, a feature selector, and the algorithm itself__. 

The data scaler we used is called __MinMaxScaler__, and essentially what it does is scales the training data onto a scale between 0 and 1, hopefully smoothing an outliers and making features more comparable. We used a feature selector called __SelectPercentile__ and what that does is takes a certain percentage of the top features in our data that contribute to the outcome, dropping the rest. We do this in attempt to get rid of features that simply add noise to our prediction, instead of any actual predictive value. Finally, as previously stated the algorithms we are using are __K Nearest Neighbors (KNN) and Decision Trees__.

#### Grid Search and Cross Validation

Now that we have a pipeline set up and ready to go, we implement what is called a __Grid Search__, and essentially what grid search does is actually carries out the act of tuning our _hyperparameters_ to create the most optimized model possible. We've used hyperparameters a few times now but here we will formally define them. __Hyperparameters__ are parameters that govern what variations of the algorithm we will be using, for example what number of neighbors to use or what upper percentile of features to keep. By inputting hyperparameters into a grid search, the grid search will _tune_, or find the best possible combination of these hyperparameters. We will discuss the specific hyperparameters used for each respective algorithm later. Grid search also utilizes a process called __Cross Validation__. Cross validation is a process in which we further split the training data, again into a smaller training data subset, and validation data, with validation data being used to assess the specific hyperparameters being tested. Cross validation doesn't make just one split, however, it splits the data multiple times, such that each subset of the larger training data set is used as validation data. Once specify which hyperparameters we want to test in a parameter grid, we can input the parameter grid into our grid search and utilize the pipeline to find the best possible algorithm for prediction. 

Now that we've detailed our machine learning process, let's talk about the specific algorithms we used.


### K Nearest Neighbors

#### Hyperparameters

Our parameter grid for our KNN model included three hyperparameters: the number of neighbors for KNN, the weighting of distance between neighbors for KNN and the upper percentile to select for feature selection. Ultimately the best combination we found was having 2 neighbors, a weight computed by how far away points were, meaning the further away a point, the less weight it had, and an upper feature selection percentile of 10%. 

#### Prediction Performance

In terms KNN's prediction performance it did relatively well. The prediction had an r-squared of 0.998, meaning that the model accounted for 99.8% of the variation in the testing data, not bad! Its mean absolute error was 0.067, meaning that on average, our model's prediction was off by 0.067 minutes when trying to predict the average hourly delay. Finally we also computed the explained variance score, which shows how far off our model was from the actual data, with the best possible value being 1.0 and the worst possible value being 0.0. The explained variance score was 0.998, indicating that the KNN model predicted the majority of variance in the actual data.


### Decision Tree

#### Hyperparameters

Our parameter grid for our decision tree model included four hyperparameters: again the upper percentile of features to select for feature selection, the number of features to consider before splitting the tree further, the maximum depth of the tree, and the minimum number of samples required to constitute a leaf node. Ultimately the best combination was a upper feature selection percentile of 5%, a maximum tree depth of 9 levels, considering all features before splitting and a minimum number of samples per leaf node of 1. 

#### Prediction Performance

Our decision tree algorithm performed extremely similar to our KNN algorithm. It had an r-squared value of 0.999 meaning our model accounted for 99.9% of the variation in the testing data, nearly perfect! Its mean absolute error was 0.044, meaning that on average, our model's prediction was off by 0.044 minutes when trying to predict the average hourly delay. Finally, the explained variance score was 0.999, showing the decision tree model predicted almost perfectly the variance in the testing data. 


#### Visualizing Model Predictions & Performance


![title](img/ml_1.png)


![title](img/ml_2.png)



## Interpretation

## Works Cited

“A Simple Formula for Estimating Evaporation Rates in Various Climates, <br> 
&emsp;Using Temperature Data Alone.” NeuroImage, Academic Press, 14 May 2003, <br>
&emsp;www.sciencedirect.com/science/article/pii/0002157177900073?via=ihub.

“Dengue Fever.” Mayo Clinic, Mayo Foundation for Medical Education and <br>
&emsp;Research, 16 Feb. 2018, www.mayoclinic.org/diseases-conditions/dengue-<br>
&emsp;fever/symptoms-causes/syc-20353078.
    
“Entomology & Ecology | Dengue | CDC.” Centers for Disease Control and <br>
&emsp;Prevention, Centers for Disease Control and Prevention, 15 Jan. 2019, <br>
&emsp;www.cdc.gov/dengue/entomologyecology/index.html.

“Frequently Asked Questions | Dengue | CDC.” Centers for Disease Control and <br>
&emsp;Prevention, Centers for Disease Control and Prevention, 15 Jan. 2019, <br>
&emsp;www.cdc.gov/dengue/faqfacts/index.html.
    
“Single Imputation Methods.” Iris Eekhout | Missing Data, <br>
&emsp;www.iriseekhout.com/missing-data/missing-data-methods/imputation <br>
&emsp;methods/.
    
“Zika Virus.” Centers for Disease Control and Prevention, Centers for <br>
&emsp;Disease Control and Prevention, 23 Feb. 2018, <br>
&emsp;www.cdc.gov/zika/vector/range.html.